<p align="center">
    <img src="https://drive.google.com/uc?id=1dFgNX9iQUfmBOdmUN2-H8rPxL3SLXmxn" width="400"/>
</p>

---

<p align="center">
<img src="https://pytorch.org/assets/images/pytorch-logo.png" alt="drawing" width="100"/>
</p>



<h1 style="text-align: center;"> Pytorch for Deep Learning Part I</h1>



In this introduction to PyTorch for deep learning, we will dive into certain packages and utilities of the library made to facilitate data handling and training opertations. We will explore similar concepts that were seen in the previous lectures, but in a different way. We will specifically focus on the functions of ``Dataset`` and ``Dataloader`` objects and their roles, also exploring adjacent functionlities for data splitting, pre-processing, and analysis. We will also discuss the basic training workflow and how ``PyTorch`` integrates these different steps. We must consider that ``PyTorch`` is an extensive library, and these lectures are designed to introduce you to its main concepts, particularly the ones that are useful for the completion of this module. With the links and references below, we encourage you to further explore the library as you see fit.


#### **Morning contents/agenda**
1. Data Handling with ``Torchvision``

2. Batch Handling with ``Dataloader``

3. Review of the ``Pytorch`` training workflow for a classification task

4. Customisation of ``Dataset`` classes


#### **Afternoon contents/agenda**

1. Understanding the basics:
- [But what is a convolution?](https://www.youtube.com/watch?v=KuXjwB4LzSA&ab_channel=3Blue1Brown)

- [But what is a neural network?](https://www.youtube.com/watch?v=aircAruvnKk&t=1s&ab_channel=3Blue1Brown)

- [What is backpropagation really doing?](https://www.youtube.com/watch?v=Ilg3gGewQ5U&t=2s&ab_channel=3Blue1Brown)

2. Exercise 2

#### **Learning Outcomes**

1. Get acquainted with common ``Torchvision`` datasets and integrate them into a ``Pytorch`` workflow

2. Learn the basic usage of ``torchvision.transforms`` 

3. Understand the difference between a ``Dataset`` and a ``Dataloader`` and how to integrate them

4. Understand the ``Pytorch`` workflow for training a classification model

5. Learn to create custom ``Dataset`` classes

7. Learn to access the ``Pytorch`` documentation

<br/>

---

<br/>

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
except:
  pass

In [ ]:
!pip install torch-summary progressbar2 -q

import torch
import torch.nn as nn
import numpy as np
import random

import matplotlib.pyplot as plt

def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = True  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = True

    return True


def set_device(device="cpu", idx=0):
    if device != "cpu":
        if torch.cuda.device_count() > idx and torch.cuda.is_available():
            print("Cuda installed! Running on GPU {} {}!".format(idx, torch.cuda.get_device_name(idx)))
            device="cuda:{}".format(idx)
        elif torch.cuda.device_count() > 0 and torch.cuda.is_available():
            print("Cuda installed but only {} GPU(s) available! Running on GPU 0 {}!".format(torch.cuda.device_count(), torch.cuda.get_device_name()))
            device="cuda:0"
        else:
            device="cpu"
            print("No GPU available! Running on CPU")
    return device

device = set_device("cuda")

## Data Handling with Torchvision's KMNIST

[``Torchvision``](https://pytorch.org/vision/stable/index.html) is one of many support libraries for the ``Pytorch`` project. It provides a range of popular models (pre-trained or not) and datasets, as well as image transformations that are useful for computer vision problems. Because it is a support library, its integration to ``Pytorch`` is native and straigth-forward, as we will see below.

In [ ]:
import torchvision

train_dataset = ###
test_dataset = ###

In [ ]:
print(type(train_dataset), "\n")
print(train_dataset, "\n")
print(dir(train_dataset), "\n") # Information held in the dataset object

In [ ]:
class_to_idx = ###
print(class_to_idx)

* The method ``__getitem__`` allows for indexing of samples within Pytorch Datasets

* For this dataset class, ``__getitem__`` returns two variables: the image and its target label

In [ ]:
sample =  ###
print(sample, "\n")

plt.imshow(###)
plt.title(###)
plt.show(###)

#### Let's visualise a few samples

In [ ]:
set_seed(42)
idxs = torch.randint(low=0, high=len(train_dataset), size=(6,))
fig, axs = plt.subplots(1, 6, figsize=(15, 5))
for i, idx in enumerate((idxs)):
    img, target = ###
    axs[i].imshow(###)
    axs[i].set_title(str(target) + " / " + class_to_idx[target])


* For convenience, let's change the format of the image from ``PIL`` to a ``torch.tensor`` object

* We will do so by using the ``torchvision.transforms`` module

* A list of transforms is often packaged into a ``Compose`` container

* Pytorch's standard practice is to apply such transformations inside the method ``__getitem__``

* It doesn't modify the data itself, but it will be applied whenever the dataset is indexed


* This circumvents the need of using pre-processing scripts to generate and store a processed dataset

In [ ]:
from torchvision.transforms import Compose, ToTensor 
transform = Compose([ToTensor(),]) # Compose a list of transformations

train_dataset = ###
test_dataset = ###

sample = ##
img, target = sample

print(img.shape, target)

* Using a ``ToTensor`` transformation is equivalent to calling ``TensorDataset``, as seen in the previous lecture. However, ``TensorDataset`` requires all data to be loaded into memory, which is not often feasible. Using the transformation allows the data to be transformed into ``torch.tensor`` objects as the data is accessed.

* Other transformations from the ``torch.transforms`` module can be found in its documentation [here](https://pytorch.org/vision/0.9/transforms.html)

* These include ``Normalize``, ``Pad``, ``RandomCrop``, ``Resize``, etc..

* There is even an option to create custom transforms with ``transforms.Lambda``. Let's have a look

In [ ]:
from torchvision.transforms import Lambda

def add_noise(x, alpha=0.1):
    return x + alpha*torch.rand_like(x)


transform = Compose([ToTensor(), 
                     ###
                     ])

train_dataset = ###
test_dataset = ###

print(train_dataset)

In [ ]:
set_seed(42)
idxs = torch.randint(low=0, high=len(train_dataset), size=(6,))
fig, axs = plt.subplots(1, 6, figsize=(15, 5))
for i, idx in enumerate((idxs)):
    img, target = ###
    axs[i].imshow(img[0], cmap="gray") # imshow expects W x H x C format
    axs[i].set_title(target)

* For now, all we need is the simple transformation of the dataset to ``torch.Tensor`` objects to allow ``PyTorch`` handling, and the standardisation of the data

In [ ]:
print("min/max:", ###)
print("mean: ", ###)
print("std: ", ###)

In [ ]:
from torchvision.transforms import Normalize 

transform = Compose([ToTensor(), # In this transformation the data is scaled from [0, 1], see documentation
                     Normalize(0.19176216423511505, 0.3483428359031677), # This transforms applies a Z-score normalisation
                     ]) 

train_dataset = ###
test_dataset = ###

print(train_dataset)

In [ ]:
print("min/max:", ###)
print("mean: ", ###)
print("std: ", ###)

* Why is the data not standardised?

## Batch Handling with ``DataLoader``

* The process of training in Pytorch relies on batch management of a particular dataset

* This is done using the [``Dataloader``](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) module, which is responsible for accessing a [``Dataset``](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset) derived object and handling it according to our needs

* In a way, the ``Dataloader`` and ``Dataset`` objects complement one another. The ``Dataset`` object handles **data** processing, whereas the ``Dataloader`` handles **batch** processing

In [ ]:
batch_size = ### # The batch size
num_workers = ### # Subprocess for loading the data

train_loader = ###

print(train_loader)
print(train_loader.__dict__) # Information held by the dataloader


#### "The ``DataLoader`` object combines a dataset and a sampler, and provides an iterable over the given dataset"

In [ ]:
set_seed(42)
train_batch_sample, train_batch_targets = ### # syntax for directly iterating over the data loader
print(train_batch_sample.shape, train_batch_targets.shape)

# Visualise batch
from torchvision.utils import make_grid

batch_grid = ###
plt.figure(figsize=(10,10))
plt.imshow(batch_grid[0], cmap="gray") # index because make_grid transforms grayscale images to RGB
plt.xticks([])
plt.yticks([])
plt.show()

print(train_batch_targets)

* Fetching the next batch of the ``train_loader`` iterator returns 32 samples and 32 corresponding target labels


In [ ]:
print("min/max:", ###)
print("mean: ", ###)
print("std: ", ###)

## Training workflow: ``Dataset`` and  ``Dataloader`` in Perspective

* Let's begin by splitting our training dataset into training and validation using ``StratifiedShuffleSplit`` and ``Subset``

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
shuffler = ###
train_idxs, valid_idxs = [(train_idx, valid_idx) for train_idx, valid_idx in shuffler][0]

# Split data with Subset (does not require data to be loaded into memory, it limits the range of which __getitem__ can be called)
from torch.utils.data import Subset
valid_dataset = ###
train_dataset = ###

# Confirm split ratio and no overlapping idxs
print("train ratio:", ###)
print("valid ratio:", ###)
print("no overlapping values", ###)

In [ ]:
print(type(train_dataset))
print(train_dataset.__dict__)

* ``Subset`` is one of the many useful tools created by Pytorch to manipulate a ``Dataset`` object without creating copies. Other operations include concatenation and chaining, and can be found in the documentation [here](https://pytorch.org/docs/stable/data.html)

* We now need two dataloaders

In [ ]:
batch_size = 64 # The batch size
num_workers = 0 # Subprocess for loading the data

train_loader = ###

valid_loader = ###

#### Let's now return to the simple feed-forward network from the previous lecture, with some custom modifications


In [ ]:
class simpleFFN(nn.Module):
  def __init__(self, ###):
    super(simpleFFN, self).__init__()
    self.hidden_1 = nn.Linear(###, ###, bias=False)
    self.hidden_2 = nn.Linear(###, ###, bias=False)
    self.output = nn.Linear(###, ###, bias=False)
    self.activation = nn.Sigmoid()
    
  def forward(self, X):
    z1 = self.hidden_1(X)
    a1 = self.activation(z1)
    z2 = self.hidden_2(a1)
    a2 = self.activation(z2)
    z3 = self.output(a2)
    a3 = self.activation(z3)
    return a3



In [ ]:
set_seed(42)
model = ###

# Check number of parameters
nparam_layer1 = ###
nparam_layer2 = ###
nparam_layer3 = ###
print(nparam_layer1 + nparam_layer2 + nparam_layer3)

* More conveniently:

In [ ]:
sum(###)  #.parameters() is a method inherited from the nn.Module base class

* Even more conveniently:

In [ ]:
from torchsummary import summary
summ = 

In [ ]:
# Test model input and output sizes with batch sample
x = ###
y = model(x).to(device)
print(x.shape, y.shape)

### The Training Workflow: Forward, Loss, Backpropagate, Optimise, Repeat

* Now that we have our data loaders ready to be iterated, let's revisit the training workflow from the last lecture

* The general steps for training a model are:
    1. Peform a foward pass of the **batch** throught the model to get initial outputs

    2. Compare the outputs with the desired targets using a difference ``metric`` to obtain a ``loss`` measure

    3. Compute derivatives of all model weights and biases with respect to the loss using back propagation
    
    4. Take an optimisation step and repeat for next batch


* Once the network goes through all batches, an ``epoch`` is concluded


* Let's use [``Adam``](https://arxiv.org/abs/1412.6980) as optimiser and [``Cross-Entropy``](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) as our metric


* First we define our ``loss``

In [ ]:
criterion = ###
print(criterion.__dict__)

* The optimiser is responsible for navigating the loss space towards the gradient direction, but it is **NOT** responsible for computing these gradients

* Commonly, these gradients are calculated using the ``.backward()`` method, which makes use of automatic differention and the computational graph to perform the operation (we will cover this in the next lecture)

* The gradients computed are stored within each ``torch.tensor`` that defines a trainable parameter. In our case, this refers to each model parameter. At each optimisation step, the optimiser updates the values of such tensors in their gradient direction for a distance specified by the learning rate, along with other parameters (e.g. momentum, weight decay)

* When constructing an optimsier, the two important things to specify are: 1) which parameters to optimise, and 2) the learning rate

In [ ]:
optimiser = ###

In [ ]:
first_param_layer = ###
print(first_param_layer.shape)
print(first_param_layer)

### Training and Validation Loops

In [ ]:
from sklearn.metrics import accuracy_score
import torch.nn.functional as F

def train(model, optimizer, criterion, data_loader):
    ###                         
    train_loss, train_accuracy = 0, 0     
    for input, target in data_loader:                       # Iterate over the mini-batches defined in the data loader
        input, target = input.to(device), target.to(device) 

        optimizer.zero_grad()                               # Resetting gradients so they don't accumulate for each .backward() pass
        
        output = ###                               # Forward pass, evaluation of model
        loss = ###                    # Compute loss
        ###                                    # Backpropagation to calculate the gradients of every parameter 
                                                            # involved in calculating this loss. Gradients are stored within tensors
        
        
        train_loss += loss*input.size(0)                    # Loss is averaged throughout batch, we scale back to later compute the 
                                                            # averaged loss for each epoch

        pred = ###          # Transform last-layer activation to probabilities, select highest value as prediction

        train_accuracy += accuracy_score(target.cpu().numpy(), pred.detach().cpu().numpy())*input.size(0) 
                                                            # Compute accuracy
                                                            # Note that we didn't call .backward() on our accuracy
                                                            # It is not a metric we are optimising for
                                                        
        
        ###                                    # Perform an optimisation step for all parameters and learning rate
                                                            # defined in the construction of the optimiser 

    train_loss = train_loss / len(data_loader.dataset)      # Average loss over the whole dataset
    train_accuracy = train_accuracy/len(data_loader.dataset)
    return train_loss, train_accuracy  


def valid(model, criterion, data_loader):
    " Equivalent to the training function without any backpropagation or optimisation steps"
    ###                        
    valid_loss, valid_accuracy = 0, 0
    with ###:     
        for input, target in data_loader:                     
            input, target = input.to(device), target.to(device) 
            
            output = model(input)                               
            loss = criterion(output, target)                                     
            
            valid_loss += loss*input.size(0)                   

            pred = output.softmax(dim=1).max(dim=1)[1]         

            valid_accuracy += accuracy_score(target.cpu().numpy(), pred.detach().cpu().numpy())*input.size(0) 

        valid_loss = valid_loss / len(data_loader.dataset)
        valid_accuracy = valid_accuracy/len(data_loader.dataset)
        return valid_loss, valid_accuracy  


In [ ]:
from progressbar import ProgressBar

nepochs = 5
with ProgressBar(max_value=nepochs) as bar:
    for i in range(nepochs):
        train_loss, train_accuracy = ###
        valid_loss, valid_accuracy = ###

        log = {"train_loss": train_loss.item(), "train_accuracy": train_accuracy.item(), 
                "valid_loss": valid_loss.item(),  "valid_accuracy": valid_accuracy.item()}
        print(log)

        bar.update(i)


* What does the final layer of our model look like?

In [ ]:
set_seed(42)
output = model(###)
plt.figure(figsize=(5,5))
plt.imshow(###, cmap="plasma", vmin=0., vmax=1.)
plt.xticks([i for i in range(output.shape[1])])
plt.xlabel("Labels")
plt.ylabel("Samples")
plt.colorbar()
plt.show()

* The plot above shows that the final layer of our trained model shows the level of "certainty" over a predicted label. Some are a clear choice, but some others cast some doubt.

---

Some considerations:

* ``model.train`` and ``model.eval`` control some underlying behaviours of the model that require different behaviours during training and inference time. For instance, ``batch normalisation`` layers no longer are activated for batch-statistics and ``dropout`` layers are deactivated (more on theses concepts in the upcoming lectures)

* ``torch.no_grad`` disables any gradient computation in ``torch.Tensors`` temporarily. Practically, this means more efficiency in inference since the computational graph of operations does not have to be retained.

* ``.detach()`` returns a new tensor that is detached from the current computational graph (gradients can no longer be computed). This is particularly useful for processing the tensor with other libraries (e.g. numpy, scikit-learn, etc)

## Deeper into ``Datasets``

* So far we have handled well-formated datasets that align very well with the ``torch`` library

* But how can we handle datasets that we might find online, or develop ourselves, that require different formating structures?

* ``ImageFolder`` is one useful base class provided by ``torchvision`` that relies on the following folder structure:

```
root/class_x/xxx.png
root/class_x/xxy.png
root/class_x/[...]/xxz.png

root/class_y/123.png
root/class_y/nsdf3.png
root/class_y/[...]/asd932_.png

```

In [ ]:
%%html
<iframe src="https://pytorch.org/vision/stable/generated/torchvision.datasets.ImageFolder.html" width="600" height="700"></iframe>

* We will use a new dataset that is available on the [Kaggle](http://www.kaggle.com) platform

* To run the following code you will need a Kaggle account and an authentication ``json`` file

* To do so follow the 'Authentication' instructions on this link https://www.kaggle.com/docs/api

* Once you downloaded the ``kaggle.json`` file, upload it to ``MyDrive``

* You can use the snippet below to retrieve datasets from Kaggle in other projects as well

In [ ]:
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle

!kaggle datasets download -d gpiosenka/butterfly-images40-species
!mkdir ./butterfly-images40-species
!unzip -q butterfly-images40-species.zip -d ./butterfly-images40-species

In [ ]:
from torchvision.datasets import ImageFolder

butterfly_train_dataset = ###

print(butterfly_train_dataset, "\n")
print(butterfly_train_dataset.class_to_idx)

butterfly, target = butterfly_train_dataset[42]
plt.imshow(butterfly)
plt.title(list(butterfly_train_dataset.class_to_idx)[target])
plt.show()

* But we can also create our own dataset class to adapt to any data structure that we'd like.

* We can do so by taking advantage of the ``Dataset`` class in ``PyTorch``, which can be used as an base class for custom datasets

* From the documentation: 

```
torch.utils.data.Dataset is an abstract class representing a dataset. Your custom dataset should inherit Dataset and override the following methods:

__len__ so that len(dataset) returns the size of the dataset.

__getitem__ to support the indexing such that dataset[i] can be used to get iith sample.

```

* Both methods are used by the ``Dataloader`` to sample the dataset. ``__getitem__`` returns a sample, and ``__len__`` defines the range in which ``__getitem__`` can be called.

---

* Let's now consider the same buttfly dataset, but instead of a classifiction problem, we are interested in a reconstruction one: deblurring. 

* We design our problem such that our input is a blurred version of our target.

In [ ]:
from torch.utils.data import Dataset
import os, glob
from PIL import Image
from torchvision.transforms import GaussianBlur

###

  # def __str__(self):
  #     class_string = ""
  #     class_string += self.__class__.__name__
  #     class_string+="\n\tlen : %d"%self.__len__()
  #     for key, value in self.__dict__.items():
  #         if key != "data_paths":
  #             class_string+="\n\t" + str(key) + " : " + str(value)
  #     return class_string

In [ ]:
butterfly_train_dataset = ###
print(butterfly_train_dataset)
input, target = ###
###

## Further considerations

* The options for customisation in ``Pytorch`` library are extensive. Mostly, you will **not** need to fully understand these concepts for a successful completion of this course.

* The documentation of ``Pytorch`` is full of examples, tutorials and paper references and will be your best reference to learn new application concepts and how to implement them.

* A few other considerations before we wrap up:

Mathematical Utils in ``Pytorch``

---
[``torch.linalg``](https://pytorch.org/docs/stable/linalg.html)
    - [``torch.fft``](https://pytorch.org/docs/stable/fft.html)
    - [``torch.random``](https://pytorch.org/docs/stable/random.html)
    - [``torch.sparse``](https://pytorch.org/docs/stable/sparse.html)
    - ...


Losses and Activations

---

The loss function drives the optimisation of machine learning models. It tells the model what it is meant to learn. In this sense, different tasks require different loss functions. In our examples so far, we have used ``CrossEntropy`` as our loss, which is well suited to multi-taks classification problems. In contrast, this loss function would not be suited, for example, to reconstruction problems. In this case we would require a loss function that would compare an image output to a target output. The ``MSELoss`` would be more suitable for this scenario. A wide range of loss functions are provided by ``Pytorch``, but it is up to you to decide which is more appropriate to your problem.






In [ ]:
%%html
<iframe src="https://pytorch.org/docs/stable/nn.html#loss-functions" width="700" height="500"></iframe>


Learning Rate Annealing

---

Learning rate annealing refers to decreasing the learning rate during the course of optimisation. This has been shown to help with convergence and stabilisation of your model, while also cutting training time. In ``Pytorch`` the implementation of learning rate annealing follows a similar syntax to the implementation of the optimiser, with different annealing strategies offered.


In [ ]:
%%html
<iframe src="https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate" width="700" height="500"></iframe>


Dataloader sampler

---

As with custom ``Dataset`` classes, we can also design better sampling strategies that are better suited to our dataset. This is an interesting way of adding or removing biases in our model. For instance, using a standard sampler for a dataset that has different amounts of data per class will bias our model to perform better to the classes that contain more samples. We can instead use a ``WeightedSampler`` to increase the probability of a weaker class to show up in our batches. As per usual, ``Pytorch`` offers a wide range of sampling strategies that can be passed into the ``Dataloader``, including a base ``Sampler`` class so we can customise our own.


In [ ]:
%%html

<iframe src="https://pytorch.org/docs/stable/data.html#torch.utils.data.Sampler" width="700" height="500"></iframe>


Model Initialisations

---

The initialisation of weights in our model can have significant impact on its convergence and performance. It is natural that ``Pytorch`` will also offer different ways to initialise the model weights. For most layers, the default initialisation method is the ``Kaiming Uniform``, which is described in the documentation:




In [ ]:
%%html
<iframe src="https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.kaiming_uniform_" width="700" height="500"></iframe>